In [1]:
import os
import sys
import pyperclip
import pandas as pd

sys.path.append('/home/fboehning/fboehning')
#sys.path.append('C:/Users/felix/Documents/Studium Ingenieurinformatik/dAISY/fboehning')

import logging
logging.basicConfig(
    level=logging.INFO,
    #level=logging.DEBUG,
    format='%(asctime)s - %(filename)s - %(name)s - %(funcName)s - %(levelname)s - %(message)s')

In [78]:
TOOLKIT=[
    'beesup_llm.toolkit.llm_utils',
    'beesup_llm.toolkit.dict_utils',
    'beesup_llm.toolkit.setup_utils',
    'beesup_llm.toolkit.visualization',
    'beesup_llm.toolkit.display_settings',
]

DATASET=[
    'beesup_llm.dataset',
]

MODEL_PIPELINES=[
    'beesup_llm.model_pipelines'
]

INJECTION=[
    'beesup_llm.injection',
    'beesup_llm.injection.taxomizing_pipeline',
    'beesup_llm.injection.taxomizing_utils',
    'beesup_llm.injection.evaluation_pipeline',
    'beesup_llm.injection.evaluation_utils'
]

EXPERIMENT=[
    'beesup_llm.experiment',
    'beesup_llm.experiment.injection_experiment'
]

import importlib
import beesup_llm;
import beesup_llm.toolkit.llm_utils; from beesup_llm.toolkit.llm_utils import *
import beesup_llm.toolkit.dict_utils; from beesup_llm.toolkit.dict_utils import *
import beesup_llm.toolkit.setup_utils; from beesup_llm.toolkit.setup_utils import *
import beesup_llm.toolkit.visualization; from beesup_llm.toolkit.visualization import *
import beesup_llm.toolkit.display_settings; from beesup_llm.toolkit.display_settings import *

import beesup_llm.dataset; from beesup_llm.dataset import *
import beesup_llm.training; from beesup_llm.training import *
import beesup_llm.model_pipelines; from beesup_llm.model_pipelines import *

# import beesup_llm.extraction; from beesup_llm.extraction import *
# import beesup_llm.extraction.extraction_utils; from beesup_llm.extraction.extraction_utils import *
# import beesup_llm.extraction.extraction_pipeline; from beesup_llm.extraction.extraction_pipeline import *

# import beesup_llm.extraction.evaluation_utils; from beesup_llm.extraction.evaluation_utils import *
# import beesup_llm.extraction.evaluation_pipeline; from beesup_llm.extraction.evaluation_pipeline import *

# import beesup_llm.experiment; from beesup_llm.experiment import *
# import beesup_llm.experiment.extraction_experiment; from beesup_llm.experiment.extraction_experiment import *

import beesup_llm.injection; from beesup_llm.injection import *
import beesup_llm.injection.taxomizing_utils; from beesup_llm.injection.taxomizing_utils import *
import beesup_llm.injection.taxomizing_pipeline; from beesup_llm.injection.taxomizing_pipeline import *


def reimport(pathlist=TOOLKIT+DATASET+MODEL_PIPELINES+INJECTION):

    pathlist = ['beesup_llm'] + pathlist

    for module_path in pathlist:
        if module_path in sys.modules:
            # Reload if the module is already imported
            importlib.reload(sys.modules[module_path])
        else:

            importlib.import_module(module_path)
            # Import the module or submodule for the first time
            globals()[module_path.split('.')[0]] = importlib.import_module(module_path)

        module = sys.modules[module_path]
        globals().update({name: getattr(module, name) for name in dir(module) if not name.startswith('_')})

import logging
def set_info():
    logger = logging.getLogger('beesup_llm')
    logger.setLevel(logging.INFO)
    logging.getLogger().setLevel(logging.INFO)

def set_debug():
    logger = logging.getLogger('beesup_llm')
    logger.setLevel(logging.DEBUG)
    logging.getLogger().setLevel(logging.DEBUG)

set_info()
reimport()

In [31]:
reimport()

In [8]:
#Load Model
llm_pipe=LlamaPipeline()
model=llm_pipe.get_model()

2025-02-03 09:16:52,410 - __init__.py - beesup_llm - __init__ - INFO - 0001_LLM_PIPELINE initialised
2025-02-03 09:16:52,414 - __init__.py - beesup_llm - load_model - INFO - Loading model meta-llama/Meta-Llama-3.1-8B-Instruct
2025-02-03 09:16:53,481 - modeling.py - accelerate.utils.modeling - get_balanced_memory - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
llm_pipe=LlamaPipeline(model)
llm_pipe.prepare_inference()

2025-02-03 09:18:57,875 - __init__.py - beesup_llm - __init__ - INFO - 0001_LLM_PIPELINE initialised


# Generate Taxonomy

In [28]:
BaseDataset.get_overview(keypaths=['emb_model_config.task','remarks','emb_model_config.name_or_path'])

,id,name,remarks,task,name_or_path
0,1,0001_dataset,"basic knowledge_df from wildbienen_db, no embe...",NaN,NaN
1,2,0002_dataset,"knowledge dataset with embeddings in subject, ...",separation,jinaai/jina-embeddings-v3
2,3,0003_dataset,"knowledge dataset with embeddings in subject, ...",classification,jinaai/jina-embeddings-v3
3,4,0004_dataset,"knowledge dataset with embeddings in subject, ...",retrieval.passage,jinaai/jina-embeddings-v3
4,5,0005_dataset,"knowledge dataset with embeddings in subject, ...",text-matching,jinaai/jina-embeddings-v3
5,6,0006_dataset,"knowledge dataset with embeddings in subject, ...",NaN,nvidia/NV-Embed-v2


In [6]:
knowledge_ds=BaseDataset(6)
knowledge_df=knowledge_ds.dataset_df
knowledge_df=knowledge_df[knowledge_df.p!='Wissenschaftlicher Name']
knowledge_df=knowledge_df.reset_index(drop=True)

def get_first_n_words(text, n=2):
    return ' '.join(re.split(r'[\s,;]',text)[:n])

knowledge_df['label']=knowledge_df.apply(lambda x: f'{get_first_n_words(x.s)}/ {x.p}', axis=1)

def compose_emb(row):
    s,p,o,so,po,spo=[row[f'{c}_emb'] for c in ['s','p','o','so','po','spo']]

    if row.attr_type=='Text':
        emb=po
    elif row.attr_type=='Attribute':
        emb=po

    # weights = np.array([0.0, 0.25, 0.75]) 
    # emb = np.average(np.array([s,p,o]), axis=0, weights=weights)
    #emb=spo-s
    #emb=po

    emb_normalized=emb/ np.linalg.norm(emb) #normalize
    return emb_normalized

chunks_df=knowledge_df.copy()
chunks_df['emb']=knowledge_df.apply(compose_emb, axis=1)
chunks_df=chunks_df[['spo','s','p','o','source_name','attr_type','n_units','n_words','label','emb']].copy()
chunks_df['label']=chunks_df.apply(lambda x: f"{get_first_n_words(x.p, n=4)}/ {get_first_n_words(x.o, n=10)}", axis=1)

chunks_df.rename(columns={
    'spo':'chunk',
    's':'subject',
    'p':'predicate',
    'o':'object',
    },inplace=True)

DROP_EMB_DUPLICATES=False
if DROP_EMB_DUPLICATES:
    chunks_df['emb_tuple']=chunks_df['emb'].apply(tuple)
    grouped = chunks_df.groupby('emb_tuple', group_keys=False).apply(lambda group: list(group.index), include_groups=False).reset_index(name='k_idc')
    chunks_df = pd.merge(chunks_df, grouped, on='emb_tuple', how='left')
    chunks_df.drop_duplicates(subset='emb_tuple',keep='first',inplace=True)
    chunks_df.drop(columns=['emb_tuple'],inplace=True)
else:
    chunks_df = chunks_df.reset_index()
    chunks_df['k_idc']=chunks_df['index'].apply(lambda x: [x])
    chunks_df.drop(columns=['index'],inplace=True)

chunks_df.reset_index(drop=True,inplace=True)
chunks_df

2025-02-03 09:15:48,771 - __init__.py - beesup_llm - __init__ - INFO - 0006_DATASET initialised


,chunk,subject,predicate,object,source_name,attr_type,n_units,n_words,label,emb,k_idc
0,"Aglaoapis tridentata (Nylander, 1848)\nAussehe...","Aglaoapis tridentata (Nylander, 1848)",Aussehen,"Die Biene ist schwarz, etwa 8–10 mm lang und m...",wikipedia,Text,1,33,Aussehen/ Die Biene ist schwarz etwa 8–10 mm ...,"[0.013315118, -0.0025690868, -0.002520601, -0....",[0]
1,"Aglaoapis tridentata (Nylander, 1848)\nFamilie...","Aglaoapis tridentata (Nylander, 1848)",Familie,Megachilidae,wildbienen-kataster,Attribute,1,1,Familie/ Megachilidae,"[-0.008217988, -0.017626682, 0.014735313, 0.00...",[1]
2,"Aglaoapis tridentata (Nylander, 1848)\nKurzbes...","Aglaoapis tridentata (Nylander, 1848)",Kurzbeschreibung,"Dunkle Zweizahnbiene (Aglaoapis tridentata, Sy...",wikipedia,Text,1,23,Kurzbeschreibung/ Dunkle Zweizahnbiene (Aglaoa...,"[0.0050212676, -0.020270795, 0.03047962, -0.01...",[2]
3,"Aglaoapis tridentata (Nylander, 1848)\nLebensr...","Aglaoapis tridentata (Nylander, 1848)",Lebensraum,"Die Biene lebt auf trockenen, blütenreichen Bö...",wikipedia,Text,1,38,Lebensraum/ Die Biene lebt auf trockenen blüt...,"[0.010719752, 0.007365582, 0.02396487, -0.0069...",[3]
4,"Aglaoapis tridentata (Nylander, 1848)\nSozialv...","Aglaoapis tridentata (Nylander, 1848)",Sozialverhalten,parasitisch,wildbienen-kataster,Attribute,1,1,Sozialverhalten/ parasitisch,"[0.0032968458, 0.008870961, 0.024923155, 0.009...",[4]
...,...,...,...,...,...,...,...,...,...,...,...
3407,"Xylocopa violacea (Linnaeus, 1758)\nNistweise\...","Xylocopa violacea (Linnaeus, 1758)",Nistweise,hypergäisch-selbstgegrabene Hohlräume,wildbienen-kataster,Attribute,1,2,Nistweise/ hypergäisch-selbstgegrabene Hohlräume,"[0.0011781659, 0.0005463787, 0.036551803, 0.00...",[3407]
3408,"Xylocopa violacea (Linnaeus, 1758)\nPollensamm...","Xylocopa violacea (Linnaeus, 1758)",Pollensammelverhalten,polylektisch,wildbienen-kataster,Attribute,1,1,Pollensammelverhalten/ polylektisch,"[0.0017042567, -0.013260176, 0.028913721, 0.00...",[3408]
3409,"Xylocopa violacea (Linnaeus, 1758)\nRequisiten...","Xylocopa violacea (Linnaeus, 1758)",Requisiten,Totholz,wildbienen-kataster,Attribute,1,1,Requisiten/ Totholz,"[0.00826123, -0.0013367897, 0.031230105, -0.00...",[3409]
3410,"Xylocopa violacea (Linnaeus, 1758)\nUmgangsspr...","Xylocopa violacea (Linnaeus, 1758)",Umgangssprachliche Bezeichnungen,Blauschwarze Holzbiene; Große Holzbiene,wildbienen-kataster,Attribute,2,4,Umgangssprachliche Bezeichnungen/ Blauschwarze...,"[0.02123162, 0.0107286265, 0.011301344, -0.010...",[3410]


In [34]:
knowledge_df

,s,p,o,source_name,attr_type,n_units,n_words,s_emb,p_emb,o_emb,spo,spo_emb,po,po_emb,so,so_emb,label
0,"Aglaoapis tridentata (Nylander, 1848)",Aussehen,"Die Biene ist schwarz, etwa 8–10 mm lang und m...",wikipedia,Text,1,33,"[0.009497706, -0.02880746, 0.008733329, 0.0057...","[-0.00241154, 0.018927356, 0.029465966, 0.0016...","[0.01207492, -0.00862774, 0.0006142589, -0.024...","Aglaoapis tridentata (Nylander, 1848)\nAussehe...","[0.017138824, -0.007862857, 0.0062451847, -0.0...","Aussehen\nDie Biene ist schwarz, etwa 8–10 mm ...","[0.013315118, -0.0025690868, -0.002520601, -0....","Aglaoapis tridentata (Nylander, 1848)\nDie Bie...","[0.015632853, -0.013067747, 0.005556736, -0.02...",Aglaoapis tridentata/ Aussehen
1,"Aglaoapis tridentata (Nylander, 1848)",Familie,Megachilidae,wildbienen-kataster,Attribute,1,1,"[0.009497706, -0.02880746, 0.008733329, 0.0057...","[-0.017521149, -0.0039982847, 0.023655187, 0.0...","[-0.009079237, -0.009580155, 0.016292965, 0.01...","Aglaoapis tridentata (Nylander, 1848)\nFamilie...","[0.005252384, -0.036279034, 0.006696027, -0.00...",Familie\nMegachilidae,"[-0.008217988, -0.017626682, 0.014735313, 0.00...","Aglaoapis tridentata (Nylander, 1848)\nMegachi...","[0.006705275, -0.036509484, 0.010870452, -0.00...",Aglaoapis tridentata/ Familie
2,"Aglaoapis tridentata (Nylander, 1848)",Kurzbeschreibung,"Dunkle Zweizahnbiene (Aglaoapis tridentata, Sy...",wikipedia,Text,1,23,"[0.009497706, -0.02880746, 0.008733329, 0.0057...","[0.005835669, 0.00046010816, 0.013336124, 0.00...","[0.0041770185, -0.026317475, 0.034765385, -0.0...","Aglaoapis tridentata (Nylander, 1848)\nKurzbes...","[0.0035359014, -0.01788473, 0.027300809, -0.01...",Kurzbeschreibung\nDunkle Zweizahnbiene (Aglaoa...,"[0.005021267, -0.020270793, 0.030479617, -0.01...","Aglaoapis tridentata (Nylander, 1848)\nDunkle ...","[0.0042278427, -0.021583805, 0.029263686, -0.0...",Aglaoapis tridentata/ Kurzbeschreibung
3,"Aglaoapis tridentata (Nylander, 1848)",Lebensraum,"Die Biene lebt auf trockenen, blütenreichen Bö...",wikipedia,Text,1,38,"[0.009497706, -0.02880746, 0.008733329, 0.0057...","[-0.00676674, 0.005776319, 0.015435509, 0.0075...","[0.007754282, 0.005550979, 0.02731938, -0.0074...","Aglaoapis tridentata (Nylander, 1848)\nLebensr...","[0.013944852, -0.0064185373, 0.027355487, -0.0...","Lebensraum\nDie Biene lebt auf trockenen, blüt...","[0.010719752, 0.007365582, 0.02396487, -0.0069...","Aglaoapis tridentata (Nylander, 1848)\nDie Bie...","[0.012453266, -0.006588385, 0.027248787, -0.01...",Aglaoapis tridentata/ Lebensraum
4,"Aglaoapis tridentata (Nylander, 1848)",Sozialverhalten,parasitisch,wildbienen-kataster,Attribute,1,1,"[0.009497706, -0.02880746, 0.008733329, 0.0057...","[0.0036254618, 0.009151482, 0.033454232, 0.006...","[0.003635382, 0.00020615666, 0.011989376, 0.01...","Aglaoapis tridentata (Nylander, 1848)\nSozialv...","[0.0068017743, -0.011348086, 0.027071316, 0.00...",Sozialverhalten\nparasitisch,"[0.0032968454, 0.00887096, 0.024923151, 0.0094...","Aglaoapis tridentata (Nylander, 1848)\nparasit...","[0.015163679, -0.020221671, 0.0124891335, 0.01...",Aglaoapis tridentata/ Sozialverhalten
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3407,"Xylocopa violacea (Linnaeus, 1758)",Nistweise,hypergäisch-selbstgegrabene Hohlräume,wildbienen-kataster,Attribute,1,2,"[0.009752467, -0.021650426, -0.007450565, 0.01...","[0.011682943, -0.005001527, 0.036846403, 0.003...","[-0.0018761744, -0.0029426122, 0.02997774, 0.0...","Xylocopa violacea (Linnaeus, 1758)\nNistweise\...","[0.010135255, -0.015556388, 0.006597145, 0.009...",Nistweise\nhypergäisch-selbstgegrabene Hohlräume,"[0.0011781659, 0.0005463787, 0.036551803, 0.00...","Xylocopa violacea (Linnaeus, 1758)\nhypergäisc...","[0.013500724, -0.011941198, 0.0052517443, 0.01...",Xylocopa violacea/ Nistweise
3408,"Xylocopa violacea (Linnaeus, 1758)",Pollensammelverhalten,polylektisch,wildbienen-kataster,Attribute,1,1,"[0.009752467, -0.021650426, -0.007450565, 0.01...","[0.006692

In [42]:
reimport()

In [43]:
tax_pipe=TaxomizingPipeline(1, llm_ref=llm_pipe)
tax_pipe.get_config()

2025-01-31 11:02:50,210 - __init__.py - beesup_llm - __init__ - INFO - 0001_TAXOMIZING_PIPELINE initialised
2025-01-31 11:02:50,221 - __init__.py - beesup_llm - __init__ - INFO - 0001_LLM_PIPELINE initialised


{'type': 'taxomizing_pipeline',
 'id': 1,
 'name': '0001_taxomizing_pipeline',
 'dir_name': 'taxomizing_pipelines',
 'lab_name': 'injection_lab',
 'rel_path': 'injection_lab/taxomizing_pipelines/0001_taxomizing_pipeline',
 'dist_flattening_config': {'include_leaves': False,
  'use_kneepoint': True,
  'use_std': False,
  'std_factor': 1.0},
 'ddist_flattening_config': {'include_leaves': False,
  'use_kneepoint': True,
  'use_std': False,
  'std_factor': 1.0},
 'linkage_args': {'method': 'ward', 'optimal_ordering': False},
 'llm_config': {'type': 'llm_pipeline',
  'id': 1,
  'name': '0001_llm_pipeline',
  'dir_name': 'llm_pipelines',
  'lab_name': 'injection_lab',
  'rel_path': 'injection_lab/llm_pipelines/0001_llm_pipeline',
  'name_or_path': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
  'pipeline_args': {'return_full_text': False,
   'clean_up_tokenization_spaces': True},
  'bnb_config': {'load_in_4bit': True,
   'bnb_4bit_use_double_quant': True,
   'bnb_4bit_quant_type': 'nf4'},
  'infe

In [44]:
tax_pipe.process(chunks_df, verbose=True)

2025-01-31 11:03:10,075 - taxomizing_utils.py - root - linkage_to_btree - INFO - {'num_nodes': 6823, 'height': 127, 'num_root_children': 2, 'num_leaves': 3412, 'branching': 2.0, 'avg_degree': 1.0, 'avg_balance': 4.32, 'dia': 236, 'avg_depth': 19.63, 'width': 656}
2025-01-31 11:03:10,252 - taxomizing_utils.py - root - get_dist_kneepoint - INFO - knee_dist: 1.0658, knee_index: 3369/ 3411
2025-01-31 11:03:10,564 - taxomizing_utils.py - root - do_dist_flattening - INFO - {'num_nodes': 3454, 'height': 9, 'num_root_children': 2, 'num_leaves': 3412, 'branching': 84.17, 'avg_degree': 1.0, 'avg_balance': 0.01, 'dia': 16, 'avg_depth': 5.24, 'width': 897}
2025-01-31 11:03:10,601 - taxomizing_utils.py - root - get_ddist_kneepoint - INFO - knee_ddist: 3.4228, knee_index: 35/ 41
2025-01-31 11:03:10,795 - taxomizing_utils.py - root - do_ddist_flattening - INFO - {'num_nodes': 3427, 'height': 4, 'num_root_children': 2, 'num_leaves': 3412, 'branching': 244.57, 'avg_degree': 1.0, 'avg_balance': 0.0, 'di

3412, 3413, 1/17                              
2/17                              8                    
3/17                              5                    
4/17                              1                    
5/17                              6                    
6/17                                                   
7/17                              3                    
8/17                              6                    
9/17                              9                    
10/17                                                
11/17                                                
12/17                                                 
13/17                                                
14/17                                                  
15/17                                                 
16/17                                                 


2025-01-31 11:04:10,929 - taxomizing_pipeline.py - beesup_llm - get_embedding_tree - INFO - all chunks included in the tree: True


[6822] Wildbienen7971622943878174                    
├──  [6820] Nist- und Bauverhalten der Wildbienen
│   ├──  [6796] Familien der Wildbienen
│   ├──  [6799] Pollensammelverhalten: Polylektizität und Oligolektizität
│   ├──  [6806] Nistplätze der Wildbienen
│   ├──  [6793] Sozialverhalten der Wildbienen
│   └──  [6807] Baumaterialien und Requisiten der Wildbienen
└──  [6821] Umgangssprachliche Bezeichnungen der Wildbienen
    ├──  [6817] Lebensweise und Merkmale
    │   ├──  [3412] Lebensweise und Nestbau


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


    │   ├──  [6798] Merkmale der Wildbienen
    │   ├──  [6797] Lebensraum und Vorkommen
    │   └──  [6801] Verbreitung und Lebensraum
    └──  [6818] Vorkommen und Umgangssprachliche Bezeichnungen der Dünen-Steppenbiene
        ├──  [3413] Umgangssprachliche Bezeichnungen der Wildbienen-Arten
        ├──  [6810] Kuckuckswespen
        └──  [6812] Wildbienenarten der Familie Apidae


# Create Instruction Finetuning Dataset

- also consider finetuning not in chat-format
- provide whole table of content in system instruction?
- multiple queries in one conversation (alternating masking required)
- consider entitie-swap.. genus species (author, year) --> german synonym 
- include source in the query prompt .. "What does wikipedia write about _predicate_ of _subject_?"

In [69]:
tax_pipe=TaxomizingPipeline(1)
tax_pipe.get_config()
tree=tax_pipe.header_tree


2025-02-03 12:00:36,738 - __init__.py - beesup_llm - __init__ - INFO - 0001_TAXOMIZING_PIPELINE initialised


In [70]:
# INSTRUCTION PROMPTING
def get_n_words_str(n_words, lang='de'):
    n_words_str=None

    if lang=='de':
        if n_words==1: n_words_str='einem Wort'
        elif n_words==2: n_words_str='zwei Wörtern'
        elif n_words==3: n_words_str='drei Wörtern'
        elif n_words==4: n_words_str='vier Wörtern'
        elif n_words==5: n_words_str='fünf Wörtern'
        elif n_words < 10: n_words_str='weniger als zehn Wörtern'
        elif n_words < 20: n_words_str='weniger als zwanzig Wörtern'
        else:
            n_words_str=f'etwa {round(n_words, -1)} Wörtern'
    
    if lang =='en':
        if n_words==1: n_words_str='one word'
        elif n_words==2: n_words_str='two words'
        elif n_words==3: n_words_str='three words'
        elif n_words==4: n_words_str='four words'
        elif n_words==5: n_words_str='five words'
        elif n_words < 10: n_words_str='less than ten words'
        elif n_words < 20: n_words_str='less than twenty words'
        else:
            n_words_str=f'approximately {round(n_words, -1)} words'
    
    return n_words_str

def get_system_prompt():
    prompt=""
    prompt+="""
You are a knowledge-oriented AI assistant. \
You provide factual knowledge about the user's input in the style of a lexicon entry, without any additional commentary, conversational language, or speculation. \
You only respond in German, using single words or brief descriptions like in an encyclopedia. \
""".strip()
    return prompt
    
def get_query_prompt(sample):

    prompt=""
    #prompt+=f"Innerhalb eines Wissensbaumes befinden wir uns an folgender Stelle:\n"
    prompt+="{toc_until}\n\n".format(**sample)

    prompt+="Was weißt du über die Eigenschaft '{predicate}' von {subject}? ".format(**sample)
    prompt+=f"Antworte mit {get_n_words_str(sample.n_words, lang='de')}!"

    return prompt

def get_prompt_messages(sample):
    
    prompt_messages=[]
    prompt_messages.append({'role':'system','content':get_system_prompt()})
    prompt_messages.append({'role':'user','content':get_query_prompt(sample)})

    return prompt_messages


In [71]:
# Create Finetuning dataset
chunks_df=tax_pipe.chunks_df.copy()
chunks_df=chunks_df.sample(100, random_state=1).copy()
chunks_df.sort_values(by='node_id',ascending=False,inplace=True)

toc_dict=dict()
for nid in chunks_df.node_id.unique():
    the_node=get_node(nid, tree)
    toc_dict[nid]=get_table_of_contents(get_tree_branch_until(the_node, tree))

chunks_df['toc_until']=chunks_df.node_id.map(toc_dict)
chunks_df['prompt_messages']=chunks_df.apply(lambda x: get_prompt_messages(x), axis=1)
chunks_df['gold_message']=chunks_df['object'].apply(lambda x: [{'role':'assistant','content':x}])
chunks_df.reset_index(drop=False, names='chunk_idx', inplace=True)
#chunks_df.sort_values(by='node_id',ascending=False,inplace=True)

# add embedding combinations
emb_cols=[f"{c}_emb" for c in ['s','p','o','so','po','spo']]
chunks_df=chunks_df.merge(knowledge_df[emb_cols], left_on='chunk_idx', right_index=True, how='left')
chunks_df

,chunk_idx,chunk,subject,predicate,object,source_name,attr_type,n_units,n_words,label,...,node_id,toc_until,prompt_messages,gold_message,s_emb,p_emb,o_emb,so_emb,po_emb,spo_emb
0,2276,"Megachile bombycina (Radoszkowski, 1874)\nPara...","Megachile bombycina (Radoszkowski, 1874)",Parasiten,Mehrere Arten der Kegelbienen nutzen die Brutz...,wikipedia,Text,1,27,Parasiten/ Mehrere Arten der Kegelbienen nutze...,...,6812,Wildbienen\n└── Umgangssprachliche Bezeichnu...,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Mehrere Art...","[0.004229834, -0.017402276, 0.005263091, 0.001...","[0.0031628946, -0.0055625094, 0.03302891, -0.0...","[0.0008278371, -0.021776352, -0.012492189, 0.0...","[0.0033396431, -0.025034204, -0.01407849, -0.0...","[0.004183899, -0.011936253, 0.0038069536, -0.0...","[0.0037486788, -0.016214078, -0.0020423068, -0..."
1,453,"Andrena nigroaenea (Kirby, 1802)\nKurzbeschrei...","Andrena nigroaenea (Kirby, 1802)",Kurzbeschreibung,Andrena nigroaenea ist eine Sandbiene aus der ...,wikipedia,Text,1,45,Kurzbeschreibung/ Andrena nigroaenea ist eine ...,...,6812,Wildbienen\n└── Umgangssprachliche Bezeichnu...,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Andrena nig...","[0.0069460585, -0.012426565, -0.005831232, 0.0...","[0.010822896, 0.015871229, 0.015933366, -2.956...","[-0.0008168921, -0.0016554035, 0.04129081, 0.0...","[0.01139046, -0.016658392, -0.0016638349, 0.00...","[0.008961742, 0.0048050727, 0.02428802, -0.007...","[0.010654636, 0.00018434554, 0.0095009925, -0...."
2,678,"Anthidiellum strigatum (Panzer, 1804)\nGefährd...","Anthidiellum strigatum (Panzer, 1804)",Gefährdung,Die IUCN listet die Art als nicht gefährdet (l...,wikipedia,Text,1,26,Gefährdung/ Die IUCN listet die Art als nicht ...,...,6812,Wildbienen\n└── Umgangssprachliche Bezeichnu...,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Die IUCN li...","[0.007597259, -0.022293532, 0.01208472, 0.0143...","[0.0033657858, 0.005752898, 0.0046650735, 0.00...","[0.0076364856, -0.022309244, 0.012178406, 0.01...","[0.009044202, -0.020516764, 0.007836375, 0.010...","[0.0083721345, -0.014640068, 0.009293911, 0.00...","[0.010003361, -0.011511814, 0.004160858, 0.003..."
3,2651,"Nomada gransassoi (Schwarz, 1986)\nKurzbeschre...","Nomada gransassoi (Schwarz, 1986)",Kurzbeschreibung,Nomada gransassoi ist eine Biene aus der Famil...,wikipedia,Text,1,10,Kurzbeschreibung/ Nomada gransassoi ist eine B...,...,6812,Wildbienen\n└── Umgangssprachliche Bezeichnu...,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Nomada gran...","[0.017768513, -0.01166132, 0.016045796, -0.000...","[-0.0052415705, -0.005533913, 0.022423591, -0....","[0.014902174, -0.0060134586, 0.021639897, -0.0...","[0.03300566, -0.0029157156, 0.018494058, -0.00...","[0.009664617, -0.0025492227, 0.016610626, -0.0...","[0.03053742, -0.0033212393, 0.022203369, -0.01..."
4,2582,"Nomada femoralis (Morawitz, 1869)\nKurzbeschre...","Nomada femoralis (Morawitz, 1869)",Kurzbeschreibung,Nomada femoralis ist eine Biene aus der Famili...,wikipedia,Text,1,10,Kurzbeschreibung/ Nomada femoralis ist eine Bi...,...,6812,Wildbienen\n└── Umgangssprachliche Bezeichnu...,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Nomada femo...","[0.0054114773, 0.00069196656, 2.0663916e-05, 0...","[-0.017521149, -0.0039982847, 0.023655187, 0.0...","[-0.009079237, -0.009580155, 0.016292965, 0.01...","[0.0003383405, 0.0001074897, -0.001843139, 0.0...","[-0.008217988, -0.017626682, 0.014735313, 0.00...","[-0.005383, -0.009349595, -0.0014068917, -0.00..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,351,"Andrena intermedia (Thomson, 1870)\nUmgangsspr...","Andrena intermedia (Thomson, 1870)",Umgangssprachliche Bezeichnungen,Berg-Kleesandbiene,wildbienen-kataster,Attribute,1,1,Umgangssprachliche Bezeichnungen/

In [72]:
dataset_df=chunks_df[['chunk_idx','prompt_messages','gold_message','toc_until','node_id']].copy()
dataset_df['split']='train'
dataset_df=dataset_df.sample(n=10, random_state=1).copy()

dataset=BaseDataset(dataset_df=dataset_df, parent_ref=6)

dataset_config=dict(
    remarks='10 instruction finetuning samples for testing',
    tax_config=tax_pipe.get_config(),
)
dataset.update_config(dataset_config)
dataset.get_config()
dataset.spawn()

2025-02-03 12:02:25,969 - __init__.py - beesup_llm - __init__ - INFO - 0007_DATASET initialised
2025-02-03 12:02:25,975 - __init__.py - beesup_llm - __init__ - INFO - 0006_DATASET initialised
2025-02-03 12:02:26,245 - __init__.py - beesup_llm - spawn_config - INFO - 0007_DATASET config spawned at injection_lab/datasets/0007_dataset
2025-02-03 12:02:26,247 - __init__.py - root - spawn - INFO - 0007_DATASET spawned at /home/fboehning/fboehning/injection_lab/datasets/0007_dataset


In [73]:
BaseDataset.get_overview(keypaths=['remarks','tax_config','split'])


,id,name,remarks,tax_config
0,1,0001_dataset,"basic knowledge_df from wildbienen_db, no embe...",NaN
1,2,0002_dataset,"knowledge dataset with embeddings in subject, ...",NaN
2,3,0003_dataset,"knowledge dataset with embeddings in subject, ...",NaN
3,4,0004_dataset,"knowledge dataset with embeddings in subject, ...",NaN
4,5,0005_dataset,"knowledge dataset with embeddings in subject, ...",NaN
5,6,0006_dataset,"knowledge dataset with embeddings in subject, ...",NaN
6,7,0007_dataset,10 instruction finetuning samples for testing,"{'type': 'taxomizing_pipeline', 'id': 1, 'name..."


In [66]:
sample=chunks_df.sample(1).iloc[0]

prompt=get_query_prompt(sample)
print(prompt)
print()
print(sample.object)

 Wildbienen
└──  Umgangssprachliche Bezeichnungen der Wildbienen
    └──  Vorkommen und Umgangssprachliche Bezeichnungen der Dünen-Steppenbiene
        └──  Wildbienenarten der Familie Apidae


Was weißt du über die Eigenschaft 'Parasiten' von Megachile bombycina (Radoszkowski, 1874)? Antworte mit etwa 30 Wörtern!

Mehrere Arten der Kegelbienen nutzen die Brutzellen von Megachile bombycina für ihren Brutparasitismus. Ein weiterer nachgewiesener Parasit ist Melittobia acasta, eine Erzwespe aus der Familie der Eulophidae.


In [15]:
prompt_messages=get_prompt_messages(sample)
response=llm_pipe(prompt_messages, stream=True)

Bombus vestalis (Geoffroy, 1785) ist eine Art der Gattung der Sandbienen (Bombus). Die Verbreitung dieser Art umfasst Europa, einschließlich Deutschland, Frankreich, Italien und Spanien. Sie ist auch in Nordafrika und im Mittelmeerraum verbreitet. Die Art bevorzugt offene Landschaften, wie Wiesen, Weiden und Felder, sowie Gebiete mit Sanddünen und Kalkstein. Sie ist eine wichtige Bestandteil der Bienenfauna in diesen Regionen. Die Art ist auch in einigen Teilen Asiens, wie China und Japan, nachgewiesen.<|eot_id|>

# Create Questions

In [74]:
dataset=BaseDataset(7)

dataset_df=dataset.dataset_df
dataset_df

2025-02-03 12:02:49,409 - __init__.py - beesup_llm - __init__ - INFO - 0007_DATASET initialised


,chunk_idx,prompt_messages,gold_message,toc_until,node_id,split
0,464,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Andrenidae'}]",Wildbienen\n└── Nist- und Bauverhalten der W...,6796,train
1,1268,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'parasitisch'}]",Wildbienen\n└── Nist- und Bauverhalten der W...,6793,train
2,393,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'endogäisch-...",Wildbienen\n└── Nist- und Bauverhalten der W...,6806,train
3,2331,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Megachilida...",Wildbienen\n└── Nist- und Bauverhalten der W...,6796,train
4,1201,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Metallische...",Wildbienen\n└── Umgangssprachliche Bezeichnu...,3413,train
5,2545,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Lasioglossu...",Wildbienen\n└── Umgangssprachliche Bezeichnu...,6810,train
6,1817,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'hypergäisch...",Wildbienen\n└── Nist- und Bauverhalten der W...,6806,train
7,1670,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Colletidae'}]",Wildbienen\n└── Nist- und Bauverhalten der W...,6796,train
8,1162,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Apidae'}]",Wildbienen\n└── Nist- und Bauverhalten der W...,6796,train
9,2979,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Megachilida...",Wildbienen\n└── Nist- und Bauverhalten der W...,6796,train


In [75]:
knowledge_df=BaseDataset(dataset.parent_config).dataset_df

# add embedding combinations
emb_cols=[f"{c}_emb" for c in ['s','p','o','so','po','spo']]
add_cols=['s','p','o','source_name','attr_type','n_units','n_words']

dataset_df=dataset_df.merge(knowledge_df[emb_cols+add_cols], left_on='chunk_idx', right_index=True, how='left')
dataset_df


2025-02-03 12:03:33,049 - __init__.py - beesup_llm - __init__ - INFO - 0006_DATASET initialised


,chunk_idx,prompt_messages,gold_message,toc_until,node_id,split,s_emb,p_emb,o_emb,so_emb,po_emb,spo_emb,s,p,o,source_name,attr_type,n_units,n_words
0,464,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Andrenidae'}]",Wildbienen\n└── Nist- und Bauverhalten der W...,6796,train,"[-0.010753225, -0.017237013, -0.00051896204, 0...","[0.010822896, 0.015871229, 0.015933366, -2.956...","[0.0025856663, -0.0038922213, 0.019023353, 0.0...","[-0.0031746894, -0.006411369, -0.0041017802, 0...","[0.008295956, 0.009364809, 0.017592456, -0.003...","[0.0011020406, -0.00017049248, 0.0042824866, -...","Andrena limata (Smith, 1853)",Umgangssprachliche Bezeichnungen,Schwarzhaarige Düstersandbiene,wildbienen-kataster,Attribute,1,2
1,1268,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'parasitisch'}]",Wildbienen\n└── Nist- und Bauverhalten der W...,6793,train,"[7.333748e-05, -0.02167897, -0.0034298624, -0....","[0.010822896, 0.015871229, 0.015933366, -2.956...","[0.0052514533, -0.009536024, 0.032819267, -0.0...","[0.016847733, -0.012956342, 0.0073041627, -0.0...","[0.013778646, 0.0047404706, 0.029050274, -0.02...","[0.0148650855, -0.0046073464, 0.018770266, -0....","Bombus rupestris (Fabricius, 1793)",Umgangssprachliche Bezeichnungen,Rotschwarze Kuckuckshummel,wildbienen-kataster,Attribute,1,2
2,393,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'endogäisch-...",Wildbienen\n└── Nist- und Bauverhalten der W...,6806,train,"[-0.006476882, -0.008703847, 0.0034661666, 0.0...","[0.010822896, 0.015871229, 0.015933366, -2.956...","[0.0051048496, 0.0017534433, 0.015987912, 0.00...","[-0.007745628, 0.0023092374, 0.0025346123, 0.0...","[0.011670619, 0.008554071, 0.011887033, -0.011...","[-0.007877628, 0.009940465, 0.008959735, -0.00...","Andrena humilis (Imhoff, 1832)",Umgangssprachliche Bezeichnungen,Gewöhnliche Dörnchensandbiene,wildbienen-kataster,Attribute,1,2
3,2331,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Megachilida...",Wildbienen\n└── Nist- und Bauverhalten der W...,6796,train,"[0.013102183, -0.013905508, 0.00020106167, 0.0...","[-0.017521149, -0.0039982847, 0.023655187, 0.0...","[-0.009761446, -0.0033340065, 0.009006581, 0.0...","[0.0027435676, -0.0126821, -0.000164679, 0.023...","[-0.017203333, -0.008402896, 0.015091042, 0.00...","[0.0023152311, -0.020977395, 0.003721734, 0.02...","Lasioglossum marginellum (Schenck, 1853)",Familie,Halictidae,wildbienen-kataster,Attribute,1,1
4,1201,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Metallische...",Wildbienen\n└── Umgangssprachliche Bezeichnu...,3413,train,"[0.0031915158, -0.018899482, -0.0014269357, 0....","[0.0036254618, 0.009151482, 0.033454232, 0.006...","[-0.006519409, 0.003931721, 0.02701271, 0.0058...","[-0.0029353688, -0.015426033, 0.017068129, 0.0...","[0.0015861867, 0.009564872, 0.035163604, 0.002...","[-0.00089925714, -0.0021927494, 0.02508419, -0...","Bombus pascuorum (Scopoli, 1763)",Sozialverhalten,sozial,wildbienen-kataster,Attribute,1,1
5,2545,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'Lasioglossu...",Wildbienen\n└── Umgangssprachliche Bezeichnu...,6810,train,"[-0.008092702, -0.010585132, 0.012372567, 0.00...","[0.0033657858, 0.005752898, 0.0046650735, 0.00...","[-0.008129387, -0.010575722, 0.012317162, 0.00...","[-0.01437777, -0.008388367, 0.007415976, 0.010...","[-0.0023588454, -0.004384505, 0.015976328, -0....","[-0.0067072758, -0.0015202365, 0.01191965, 0.0...","Lithurgus chrysurus (Fonscolombe, 1834)",Wissenschaftlicher Name,"Lithurgus chrysurus (Fonscolombe, 1834)",wildbienen-kataster,Attribute,1,3
6,1817,"[{'role': 'system', 'content': 'You are a know...","[{'role': 'assistant', 'content': 'hypergäisch...",Wildbienen\n└── Nist- und Bauverhalten der W...,6806,train,"[0.00297295, -0.012001829, 0.009464798, 0.0149...","[0.010822896, 0.0158

In [166]:
from sklearn.metrics.pairwise import cosine_similarity

def fetch_similar_samples(sample, chunks_df):

    emb_col=random.choice([f"{c}_emb" for c in ['s','p','o','so','po','spo']])
    print(f"selected emb_col: {emb_col}")

    fetch_similar_samples.emb_col=emb_col

    chunk_embs=np.vstack(chunks_df[emb_col].values)
    sample_emb=sample[emb_col]

    match_matrix=cosine_similarity(sample_emb.reshape(1,-1),chunk_embs)[0]
    match_matrix[sample.name]=1.5

    ranking_df=chunks_df[['chunk_idx','subject','predicate','object']].copy()
    ranking_df['score']=match_matrix
    ranking_df.sort_values(by='score',ascending=False,inplace=True)
    ranking_df.reset_index(drop=True,inplace=True)
    return ranking_df

## MCQ - Multiple Choice Questions

### generate multiple choice questions

In [165]:
def get_mcq_generation_prompt(samples_df):

    def get_chunk_text(chunk_row):
        chunk_template="{subject}\t{predicate}\t{object}"
        #chunk_template="{predicate}: {object}"

        chunk_text=chunk_template.format(**chunk_row)
        chunk_text=re.sub(r'\n+', '; ', chunk_text)

        return chunk_text

    prompt=""
    prompt+="""
You are a teacher and you need to create a multiple-choice evaluation question to test a students knowledge. \
I will provide one or more knowledge chunks and you need to create a multiple-choice question that evaluates the student's understanding or ability to infer based on the knowledge. \
The multiple-choise question should have four answer choices, of which at least one option should be correct. \
""".strip()
    prompt+="\n\n"
    prompt+="""
Return the output in the following format as a JSON string:
{
    "question": "Here goes the generated question in German",
    "right_choices": ["a correct answer",], # List of correct answer options, at least one/ at most three
    "wrong_choices": ["a wrong answer", "another wrong answer"] # List of wrong answer options, at least one/ at most three
}
""".strip()
    
    prompt+="\n\n"
    prompt+="### KNOWLEDGE:\n"

    chars, max_chars =0, 600
    for i,row in samples_df.iterrows():
        line=get_chunk_text(row)+'\n\n'
        prompt+=line
        chars+=len(line)
        if chars>max_chars: break
    
    get_mcq_generation_prompt.samples_df=samples_df.iloc[:i+1].copy()
    
    prompt+="\n"
    prompt+="### QUESTION:\n"

    return prompt


In [204]:
sample=chunks_df.sample(1).iloc[0]

print(f"{sample.subject} - {sample.predicate}")
print(sample.object)
print()

ranking_df=fetch_similar_samples(sample, chunks_df)
emb_col=fetch_similar_samples.emb_col

prompt=get_mcq_generation_prompt(ranking_df)

display(get_mcq_generation_prompt.samples_df)
chunk_idc=get_mcq_generation_prompt.samples_df.chunk_idx.values

print(prompt)

Hoplitis mitis (Nylander, 1852) - Pollensammelverhalten
oligolektisch

selected emb_col: spo_emb


,chunk_idx,subject,predicate,object,score
0,1626,"Hoplitis mitis (Nylander, 1852)",Pollensammelverhalten,oligolektisch,1.500000
1,1556,"Heriades crenulata (Nylander, 1856)",Pollensammelverhalten,oligolektisch,0.821977
2,403,"Andrena limata (Smith, 1853)",Pollensammelverhalten,polylektisch,0.727853
3,99,"Andrena bicolor (Fabricius, 1775)",Pollensammelverhalten,polylektisch,0.727403
4,1728,"Hylaeus duckei (Alfken, 1904)",Pollensammelverhalten,breit-oligolektisch (Apiaceae),0.714924
5,613,"Andrena synadelpha (Perkins, 1914)",Pollensammelverhalten,polylektisch,0.708192
6,1903,"Lasioglossum buccale (Pérez, 1903)",Pollensammelverhalten,polylektisch,0.703867
7,13,"Amegilla quadrifasciata (de Villers, 1789)",Pollensammelverhalten,polylektisch,0.703381
8,1897,"Lasioglossum brevicorne (Schenck, 1870)",Nistweise,endogäisch-selbstgegrabene Hohlräume,0.648227


You are a teacher and you need to create a multiple-choice evaluation question to test a students knowledge. I will provide one or more knowledge chunks and you need to create a multiple-choice question that evaluates the student's understanding or ability to infer based on the knowledge. The multiple-choise question should have four answer choices, of which at least one option should be correct.

Return the output in the following format as a JSON string:
{
    "question": "Here goes the generated question in German",
    "right_choices": ["a correct answer",], # List of correct answer options, at least one/ at most three
    "wrong_choices": ["a wrong answer", "another wrong answer"] # List of wrong answer options, at least one/ at most three
}

### KNOWLEDGE:
Hoplitis mitis (Nylander, 1852)	Pollensammelverhalten	oligolektisch

Heriades crenulata (Nylander, 1856)	Pollensammelverhalten	oligolektisch

Andrena limata (Smith, 1853)	Pollensammelverhalten	polylektisch

Andrena bicolor (Fab

In [203]:
mcq_sample=dict(
    emb_col=emb_col,
    chunk_idc=chunk_idc,
    prompt=prompt,
)
mcq_sample["completion"]="""
{
    "question": "Welche Aussage trifft auf Andrena agilissima zu?",
    "right_choices": ["Sie nistet in selbst gegrabenen Höhlen in Steilwänden aus Löss oder Sand.", "Sie ist Wirtsart der brutparasitischen Blutbiene Sphecodes rubicundus."],
    "wrong_choices": ["Sie bildet mehrere Generationen pro Jahr aus.", "Sie kann nur in unmittelbarer Nähe ihres Nests Nahrung sammeln."]
}

""".strip()

mcq_samples_df=pd.read_pickle('mcq_samples_df.pkl')
mcq_samples_df=pd.concat([mcq_samples_df, pd.DataFrame([mcq_sample])], ignore_index=True)
#mcq_samples_df.to_pickle('mcq_samples_df.pkl')
mcq_samples_df

,emb_col,chunk_idc,prompt,completion
0,p_emb,"[2943, 1176]",You are a teacher and you need to create a mul...,"{\n ""question"": ""Welche Aussage trifft auf ..."
1,p_emb,[2828],You are a teacher and you need to create a mul...,"{\n ""question"": ""Welche Merkmale treffen au..."
2,po_emb,"[3086, 3081, 161, 2000, 975, 1201, 1266]",You are a teacher and you need to create a mul...,"{\n ""question"": ""Welche Bienenart hat *Noma..."
3,s_emb,"[2980, 2979, 2963, 2943, 2928]",You are a teacher and you need to create a mul...,"{\n ""question"": ""Welche Aussage trifft auf ..."
4,o_emb,"[2928, 906, 2337, 1051, 2158, 3094, 1268, 1613]",You are a teacher and you need to create a mul...,"{\n ""question"": ""Welche Bienenart nutzt Sch..."
5,p_emb,"[893, 2572, 40]",You are a teacher and you need to create a mul...,"{\n ""question"": ""Welche Aussage trifft auf ..."
6,so_emb,"[3151, 3153, 3139]",You are a teacher and you need to create a mul...,"{\n ""question"": ""Welcher Parasit befällt di..."
7,p_emb,"[1030, 1674, 3169, 1201, 2158, 1051, 906]",You are a teacher and you need to create a mul...,"{\n ""question"": ""Welche Bienenart wird umga..."
8,o_emb,"[1903, 99, 403, 613, 13, 1626, 1556, 1268, 1728]",You are a teacher and you need to create a mul...,"{\n ""question"": ""Welche Bienenarten sammeln..."
9,po_emb,[40],You are a teacher and you need to create a mul...,"{\n ""question"": ""Welche Aussage trifft auf ..."


In [ ]:
mcq_samples_df=pd.read_pickle('mcq_samples_df.pkl')

import json
LETTERS='ABCDEFGHIJKLMNOPQRSTUVWXYZ'

mcq_samples_df[['question','right_choices','wrong_choices']]=None,None,None

for i, sample_row in mcq_samples_df.iterrows():
    mcq_dict=json.loads(sample_row['completion'])
    all_choices=mcq_dict['right_choices']+mcq_dict['wrong_choices']
    random.shuffle(all_choices, )

    question=mcq_dict['question']
    for j,choice in enumerate(all_choices):
        question+=f'\n{LETTERS[j]}) {choice}'
    
    mcq_samples_df.at[i,'question']=question
    mcq_samples_df.at[i,'right_choices']=[LETTERS[all_choices.index(choice)] for choice in mcq_dict['right_choices']]
    mcq_samples_df.at[i,'wrong_choices']=[LETTERS[all_choices.index(choice)] for choice in mcq_dict['wrong_choices']]

use_cols=['question','right_choices','wrong_choices','chunk_idc']
mcq_samples_df=mcq_samples_df[use_cols].copy()
mcq_samples_df

,question,right_choices,wrong_choices,chunk_idc
0,Welche Aussage trifft auf das Verbreitungsgebi...,"[B, A]","[C, D]","[2943, 1176]"
1,Welche Merkmale treffen auf Nomada sexfasciata...,"[C, D]","[A, B]",[2828]
2,Welche Bienenart hat *Nomada fuscicornis* als ...,[B],"[C, A, D]","[3086, 3081, 161, 2000, 975, 1201, 1266]"
3,Welche Aussage trifft auf Osmia inermis zu?\nA...,"[A, B]","[D, C]","[2980, 2979, 2963, 2943, 2928]"
4,Welche Bienenart nutzt Schneckenhäuser als Req...,[C],"[D, A, B]","[2928, 906, 2337, 1051, 2158, 3094, 1268, 1613]"
5,Welche Aussage trifft auf *Andrena agilissima*...,"[C, B]","[D, A]","[893, 2572, 40]"
6,Welcher Parasit befällt die Große Schlürfbiene...,"[A, B]","[C, D]","[3151, 3153, 3139]"
7,Welche Bienenart wird umgangssprachlich als 'M...,[A],"[D, B, C]","[1030, 1674, 3169, 1201, 2158, 1051, 906]"
8,Welche Bienenarten sammeln Pollen polylektisch...,"[B, D, C]",[A],"[1903, 99, 403, 613, 13, 1626, 1556, 1268, 1728]"
9,Welche Aussage trifft auf Andrena agilissima z...,"[A, B]","[C, D]",[40]


### query the model

In [245]:
def get_mcq_prompt(sample):
    prompt=""
    prompt+="""
You are provided with the following multiple-choice question. \
Carefully review the options and select the correct one/ the correct ones. \
Respond only with the capital letter or a concatenation of correct letters. \
Do not provide any additional explanation or reasoning.
""".strip()

    prompt+=f"\n\n### QUESTION:\n{sample.question}\n\n"
    prompt+="### LETTER/ LETTERS OF CORRECT CHOICE/ CHOICES:\n"
    return prompt

def clean_completion(completion):
    completion=completion.replace('<|eot_id|>','')
    return completion

def get_pred_completion(sample, llm_pipe):
    completion=llm_pipe(sample.prompt, use_chatformat=True, stream=False, stop_strings=['\n'])[0]['generated_text']
    completion=clean_completion(completion)
    return completion

def get_mcq_eval_dict(sample):

    gold_choices=set(sample['right_choices'])
    pred_choices=set([a.strip().upper() for a in re.split(r'[,\?;! ]',sample['pred_completion']) if (a.strip() and a in LETTERS)]) 

    choices=set(gold_choices.union(set(sample['wrong_choices'])))

    tp=len(gold_choices.intersection(pred_choices))
    fp=len(pred_choices-gold_choices)
    fn=len(gold_choices-pred_choices)
    tn=len(choices-(gold_choices.union(pred_choices)))

    pre=tp/(tp+fp) if tp+fp>0 else 0
    rec=tp/(tp+fn) if tp+fn>0 else 0
    f1_score=2*(pre*rec)/(pre+rec) if pre+rec>0 else 0

    eval_dict=dict(
        tp=tp, fp=fp, fn=fn, tn=tn,
        P=pre, R=rec, F1=f1_score
    )

    return eval_dict

mcq_samples_df['prompt']=mcq_samples_df.apply(get_mcq_prompt, axis=1)
mcq_samples_df['pred_completion']=mcq_samples_df.apply(lambda x: get_pred_completion(x, llm_pipe), axis=1)

mcq_samples_df['eval_dict']=mcq_samples_df.apply(get_mcq_eval_dict, axis=1)
mcq_samples_df = mcq_samples_df.join(pd.json_normalize(mcq_samples_df['eval_dict']))
mcq_samples_df

,question,right_choices,wrong_choices,chunk_idc,prompt,pred_completion,eval_dict,tp,fp,fn,tn,P,R,F1
0,Welche Aussage trifft auf das Verbreitungsgebi...,"[D, B]","[C, A]","[2943, 1176]",You are provided with the following multiple-c...,D,"{'tp': 1, 'fp': 0, 'fn': 1, 'tn': 2, 'P': 1.0,...",1,0,1,2,1.0,0.5,0.666667
1,Welche Merkmale treffen auf Nomada sexfasciata...,"[B, A]","[C, D]",[2828],You are provided with the following multiple-c...,A,"{'tp': 1, 'fp': 0, 'fn': 1, 'tn': 2, 'P': 1.0,...",1,0,1,2,1.0,0.5,0.666667
2,Welche Bienenart hat *Nomada fuscicornis* als ...,[B],"[C, D, A]","[3086, 3081, 161, 2000, 975, 1201, 1266]",You are provided with the following multiple-c...,A,"{'tp': 0, 'fp': 1, 'fn': 1, 'tn': 2, 'P': 0.0,...",0,1,1,2,0.0,0.0,0.000000
3,Welche Aussage trifft auf Osmia inermis zu?\nA...,"[D, B]","[C, A]","[2980, 2979, 2963, 2943, 2928]",You are provided with the following multiple-c...,D,"{'tp': 1, 'fp': 0, 'fn': 1, 'tn': 2, 'P': 1.0,...",1,0,1,2,1.0,0.5,0.666667
4,Welche Bienenart nutzt Schneckenhäuser als Req...,[A],"[B, D, C]","[2928, 906, 2337, 1051, 2158, 3094, 1268, 1613]",You are provided with the following multiple-c...,A,"{'tp': 1, 'fp': 0, 'fn': 0, 'tn': 3, 'P': 1.0,...",1,0,0,3,1.0,1.0,1.000000
5,Welche Aussage trifft auf *Andrena agilissima*...,"[B, D]","[A, C]","[893, 2572, 40]",You are provided with the following multiple-c...,B,"{'tp': 1, 'fp': 0, 'fn': 1, 'tn': 2, 'P': 1.0,...",1,0,1,2,1.0,0.5,0.666667
6,Welcher Parasit befällt die Große Schlürfbiene...,"[D, B]","[C, A]","[3151, 3153, 3139]",You are provided with the following multiple-c...,A,"{'tp': 0, 'fp': 1, 'fn': 2, 'tn': 1, 'P': 0.0,...",0,1,2,1,0.0,0.0,0.000000
7,Welche Bienenart wird umgangssprachlich als 'M...,[B],"[A, C, D]","[1030, 1674, 3169, 1201, 2158, 1051, 906]",You are provided with the following multiple-c...,B,"{'tp': 1, 'fp': 0, 'fn': 0, 'tn': 3, 'P': 1.0,...",1,0,0,3,1.0,1.0,1.000000
8,Welche Bienenarten sammeln Pollen polylektisch...,"[D, C, B]",[A],"[1903, 99, 403, 613, 13, 1626, 1556, 1268, 1728]",You are provided with the following multiple-c...,A,"{'tp': 0, 'fp': 1, 'fn': 3, 'tn': 0, 'P': 0.0,...",0,1,3,0,0.0,0.0,0.000000
9,Welche Aussage trifft auf Andrena agilissima z...,"[B, D]","[C, A]",[40],You are provided with the following multiple-c...,B,"{'tp': 1, 'fp': 0, 'fn': 1, 'tn': 2, 'P': 1.0,...",1,0,1,2,1.0,0.5,0.666667


In [26]:
reimport()

In [27]:
eval_pipe=MCQEvaluationPipeline(llm_ref=llm_pipe, df=mcq_samples_df)
eval_pipe.get_config()

2025-02-03 09:52:26,841 - __init__.py - beesup_llm - __init__ - INFO - 0001_EVALUATION_PIPELINE initialised
2025-02-03 09:52:26,863 - __init__.py - beesup_llm - __init__ - INFO - 0001_LLM_PIPELINE initialised


{'type': 'evaluation_pipeline',
 'id': 1,
 'name': '0001_evaluation_pipeline',
 'dir_name': 'evaluation_pipelines',
 'lab_name': 'injection_lab',
 'rel_path': 'injection_lab/evaluation_pipelines/0001_evaluation_pipeline',
 'llm_config': {'type': 'llm_pipeline',
  'id': 1,
  'name': '0001_llm_pipeline',
  'dir_name': 'llm_pipelines',
  'lab_name': 'injection_lab',
  'rel_path': 'injection_lab/llm_pipelines/0001_llm_pipeline',
  'name_or_path': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
  'pipeline_args': {'return_full_text': False,
   'clean_up_tokenization_spaces': True},
  'bnb_config': {'load_in_4bit': True,
   'bnb_4bit_use_double_quant': True,
   'bnb_4bit_quant_type': 'nf4'},
  'inference_tokenizer_config': {'padding_side': 'left',
   'padding': 'longest',
   'add_special_tokens': True,
   'max_length': 8192,
   'pad_token': '<|begin_of_text|>',
   'pad_token_id': 128000},
  'training_tokenizer_config': {'padding_side': 'right',
   'padding': 'longest',
   'add_special_tokens': True

In [28]:
eval_pipe.spawn()

2025-02-03 09:52:29,608 - __init__.py - beesup_llm - spawn_config - INFO - 0001_EVALUATION_PIPELINE config spawned at injection_lab/evaluation_pipelines/0001_evaluation_pipeline


In [29]:
eval_pipe=MCQEvaluationPipeline(1, llm_ref=llm_pipe)
eval_pipe.get_config()

2025-02-03 09:53:17,876 - __init__.py - beesup_llm - __init__ - INFO - 0001_EVALUATION_PIPELINE initialised
2025-02-03 09:53:17,896 - __init__.py - beesup_llm - __init__ - INFO - 0001_LLM_PIPELINE initialised


{'type': 'evaluation_pipeline',
 'id': 1,
 'name': '0001_evaluation_pipeline',
 'dir_name': 'evaluation_pipelines',
 'lab_name': 'injection_lab',
 'rel_path': 'injection_lab/evaluation_pipelines/0001_evaluation_pipeline',
 'llm_config': {'type': 'llm_pipeline',
  'id': 1,
  'name': '0001_llm_pipeline',
  'dir_name': 'llm_pipelines',
  'lab_name': 'injection_lab',
  'rel_path': 'injection_lab/llm_pipelines/0001_llm_pipeline',
  'name_or_path': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
  'pipeline_args': {'return_full_text': False,
   'clean_up_tokenization_spaces': True},
  'bnb_config': {'load_in_4bit': True,
   'bnb_4bit_use_double_quant': True,
   'bnb_4bit_quant_type': 'nf4'},
  'inference_tokenizer_config': {'padding_side': 'left',
   'padding': 'longest',
   'add_special_tokens': True,
   'max_length': 8192,
   'pad_token': '<|begin_of_text|>',
   'pad_token_id': 128000},
  'training_tokenizer_config': {'padding_side': 'right',
   'padding': 'longest',
   'add_special_tokens': True

In [30]:
eval_df=eval_pipe(llm_ref=llm_pipe)
eval_df

2025-02-03 09:53:42,042 - __init__.py - beesup_llm - __init__ - INFO - 0001_LLM_PIPELINE initialised


,question,right_choices,wrong_choices,chunk_idc,prompt,pred_completion,eval_dict,tp,fp,fn,tn,P,R,F1
0,Welche Aussage trifft auf das Verbreitungsgebi...,"[B, A]","[C, D]","[2943, 1176]",You are provided with the following multiple-c...,B,"{'tp': 1, 'fp': 0, 'fn': 1, 'tn': 2, 'P': 1.0,...",1,0,1,2,1.0,0.5,0.666667
1,Welche Merkmale treffen auf Nomada sexfasciata...,"[C, D]","[A, B]",[2828],You are provided with the following multiple-c...,A,"{'tp': 0, 'fp': 1, 'fn': 2, 'tn': 1, 'P': 0.0,...",0,1,2,1,0.0,0.0,0.000000
2,Welche Bienenart hat *Nomada fuscicornis* als ...,[B],"[C, A, D]","[3086, 3081, 161, 2000, 975, 1201, 1266]",You are provided with the following multiple-c...,C,"{'tp': 0, 'fp': 1, 'fn': 1, 'tn': 2, 'P': 0.0,...",0,1,1,2,0.0,0.0,0.000000
3,Welche Aussage trifft auf Osmia inermis zu?\nA...,"[A, B]","[D, C]","[2980, 2979, 2963, 2943, 2928]",You are provided with the following multiple-c...,A,"{'tp': 1, 'fp': 0, 'fn': 1, 'tn': 2, 'P': 1.0,...",1,0,1,2,1.0,0.5,0.666667
4,Welche Bienenart nutzt Schneckenhäuser als Req...,[C],"[D, A, B]","[2928, 906, 2337, 1051, 2158, 3094, 1268, 1613]",You are provided with the following multiple-c...,B,"{'tp': 0, 'fp': 1, 'fn': 1, 'tn': 2, 'P': 0.0,...",0,1,1,2,0.0,0.0,0.000000
5,Welche Aussage trifft auf *Andrena agilissima*...,"[C, B]","[D, A]","[893, 2572, 40]",You are provided with the following multiple-c...,C,"{'tp': 1, 'fp': 0, 'fn': 1, 'tn': 2, 'P': 1.0,...",1,0,1,2,1.0,0.5,0.666667
6,Welcher Parasit befällt die Große Schlürfbiene...,"[A, B]","[C, D]","[3151, 3153, 3139]",You are provided with the following multiple-c...,C,"{'tp': 0, 'fp': 1, 'fn': 2, 'tn': 1, 'P': 0.0,...",0,1,2,1,0.0,0.0,0.000000
7,Welche Bienenart wird umgangssprachlich als 'M...,[A],"[D, B, C]","[1030, 1674, 3169, 1201, 2158, 1051, 906]",You are provided with the following multiple-c...,A,"{'tp': 1, 'fp': 0, 'fn': 0, 'tn': 3, 'P': 1.0,...",1,0,0,3,1.0,1.0,1.000000
8,Welche Bienenarten sammeln Pollen polylektisch...,"[B, D, C]",[A],"[1903, 99, 403, 613, 13, 1626, 1556, 1268, 1728]",You are provided with the following multiple-c...,A,"{'tp': 0, 'fp': 1, 'fn': 3, 'tn': 0, 'P': 0.0,...",0,1,3,0,0.0,0.0,0.000000
9,Welche Aussage trifft auf Andrena agilissima z...,"[A, B]","[C, D]",[40],You are provided with the following multiple-c...,A,"{'tp': 1, 'fp': 0, 'fn': 1, 'tn': 2, 'P': 1.0,...",1,0,1,2,1.0,0.5,0.666667


## QDQ - Query Driven Questions

- Attribute-Based Understanding
- test the ability to connect and contextualize information from disparate chunks into a unified understanding.
- only cover knowlege with attr_type=='Attribute'
- Assesses categorization skills by identifying entities that fall into predefined groups based on their attributes.
- the answers cannot be derived from a single chunk but involve synthesizing information from several chunks
- "SQL-like" answer


In [5]:
# get attrs_df
attrs_df=knowledge_df[knowledge_df['attr_type']=='Attribute'].copy()
attrs_df=attrs_df[['s','p','o']]
attrs_df.rename(
    columns={
        's':'scientific_name',
        'p':'attribute',
        'o':'value'
        },inplace=True)

attrs_df=attrs_df.pivot(index='scientific_name', columns='attribute', values='value').reset_index()
attrs_df.columns.name=None
attrs_df = attrs_df.reset_index(drop=True)
attrs_df.fillna('', inplace=True)
attrs_df

,scientific_name,Baumaterial,Familie,Kuckucksbienen,Kuckuckswespen,Nistweise,Pollensammelverhalten,Requisiten,Sozialverhalten,Umgangssprachliche Bezeichnungen,Wirtsbienen,Wissenschaftlicher Name
0,"Aglaoapis tridentata (Nylander, 1848)",,Megachilidae,,,,,,parasitisch,Dunkle Zweizahnbiene; Zweizahnbiene,"Hoplitis anthocopoides (Schenck, 1853); Hoplit...","Aglaoapis tridentata (Nylander, 1848)"
1,"Amegilla quadrifasciata (de Villers, 1789)",,Apidae,"Thyreus histrionicus (Illiger, 1806)",,endogäisch-selbstgegrabene Hohlräume,polylektisch,,,Weiße Bindenpelzbiene,,"Amegilla quadrifasciata (de Villers, 1789)"
2,"Ammobates punctatus (Fabricius, 1804)",,Apidae,,,,,,parasitisch,Große Sandgängerbiene; Sandgänger-Biene,"Anthophora bimaculata (Panzer, 1798)","Ammobates punctatus (Fabricius, 1804)"
3,"Ammobatoides abdominalis (Eversmann, 1852)",,,,,,,,,Steppenglanzbiene,,
4,"Andrena afzeliella (Kirby, 1802)",,Andrenidae,,,endogäisch-selbstgegrabene Hohlräume,,,,Kleine Kleesandbiene; Kleine Kleesandbiene,,"Andrena afzeliella (Kirby, 1802)"
...,...,...,...,...,...,...,...,...,...,...,...,...
506,"Thyreus orbatus (Lepeletier, 1841)",,Apidae,,,,,,parasitisch,Fleckenbiene; Schwarzgesichtige Fleckenbiene,"Anthophora borealis Morawitz, 1864; Anthophora...","Thyreus orbatus (Lepeletier, 1841)"
507,"Trachusa byssina (Panzer, 1798)","Blattstücke, Harz",Megachilidae,"Coelioxys conica (Linnaeus, 1758)",,endogäisch-selbstgegrabene Hohlräume,oligolektisch,,,Große Harzbiene,,"Trachusa byssina (Panzer, 1798)"
508,"Xylocopa iris (Christ, 1791)",Mark,Apidae,,,hypergäisch-selbstgegrabene Hohlräume,polylektisch,Pflanzenstängel,,Kleine Holzbiene,,"Xylocopa iris (Christ, 1791)"
509,"Xylocopa valga (Gerstäcker, 1872)",Holzgenagsel,,,Polochrum repandum Spinola 1806,hypergäisch-selbstgegrabene Hohlräume,,Totholz,,Schwarzfühler-Holzbiene; Südliche Holzbiene; Ö...,,


In [8]:
# Query Driven Questions
QDQ_SAMPLES=[
dict(
    conditions=(
        (attrs_df.Familie=='Megachilidae')
        &(attrs_df.Pollensammelverhalten.str.contains('polylektisch'))
        &(attrs_df['Baumaterial'].str.contains('Blattstücke'))
    ),
    target_attrs=['scientific_name'],
    question='Welche Bienenarten der Megachilidae sammelen Pollen von vielen verschiedenen Pflanzenarten und bauen ihre Nester aus Blattstücken?'
),
dict(
    conditions=(
        (attrs_df.Kuckucksbienen.str.contains('Nomada panzeri'))  
    ),
    target_attrs=['Familie'],
    question='Für die Arten welcher Bienenfamilie ist die Art Nomada panzeri eine Kuckucksbiene?'
),
dict(
    conditions=(
        (attrs_df['Umgangssprachliche Bezeichnungen'].str.contains('Zwergsandbiene')) 
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche unterschiedlichen Arten der Zwergsandbiene gibt es?'
),
dict(
    conditions=(
        (attrs_df.Familie=='Halictidae')
        & (attrs_df['Nistweise'].str.contains('endogäisch'))
        & (attrs_df['Pollensammelverhalten'].str.contains('oligolektisch'))
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten der Familie Halictidae bauen ihre Nester in der Erde und sammeln Pollen von wenigen Pflanzenarten?'
),
dict(
    conditions=(
        (attrs_df['Nistweise'].str.contains('hypergäisch'))
        & (attrs_df['Baumaterial'].str.contains('Harz'))
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten bauen ihre Nester oberirdisch und verwenden Harz als Baumaterial?'
),
dict(
    conditions=(
        (attrs_df.Familie=='Halictidae')
        & (attrs_df['Umgangssprachliche Bezeichnungen'].str.contains('Furchenbiene'))
        & (attrs_df.Pollensammelverhalten=='polylektisch') 
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten der Familie Halictidae werden als Furchenbienen bezeichnet und sammeln Pollen von vielen verschiedenen Pflanzenarten?'
),
dict(
    conditions=(
        (attrs_df['scientific_name'].str.contains('Morawitz'))
        & (attrs_df['Umgangssprachliche Bezeichnungen'].str.contains('Wespenbiene'))  
    ),
    target_attrs=['Umgangssprachliche Bezeichnungen'],
    question='Welche Wespenbienen wurden durch Morawitz beschrieben?'
),
dict(
    conditions=(
        (attrs_df.Familie=='Halictidae')
        & (attrs_df['Wirtsbienen'].str.contains('Andrena'))       
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten der Familie Halictidae parasitieren die Gattung Andrena?'
),
dict(
    conditions=(
        (attrs_df['Sozialverhalten'].str.contains('komunal'))  
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten haben ein komunales Sozialverhalten?'
),
dict(
    conditions=(
        (attrs_df['Umgangssprachliche Bezeichnungen'].str.contains('Filzbiene'))  
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten werden als Filzbienen bezeichnet?'
),
dict(
    conditions=(
        (attrs_df['Kuckuckswespen'].str.contains('Sapyga'))
        & (attrs_df.Pollensammelverhalten.str.contains('polylektisch')) 
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten sammeln Pollen von vielen verschiedenen Pflanzenarten und haben Kuckuckswespen der Gattung Sapyga?'
),
dict(
    conditions=(
        (attrs_df['Kuckuckswespen'].str.contains('Sapyga'))
        & (attrs_df.Pollensammelverhalten.str.contains('oligolektisch')) 
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten sammeln Pollen von wenigen Pflanzenarten und haben Kuckuckswespen der Gattung Sapyga?'
),
dict(
    conditions=(
        (attrs_df['Baumaterial'].str.contains('Lehm'))
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten verwenden unter anderem Lehm als Baumaterial?'
),
dict(
    conditions=(
        (attrs_df['Nistweise'].str.contains('Insektenfraßgänge'))
        &(attrs_df['Baumaterial'].str.contains('Blattstücke'))
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten bauen ihre Nester in Insektenfraßgängen und verwenden Blattstücke als Baumaterial?'
),
dict(
    conditions=(
        (attrs_df['Nistweise'].str.contains('Mauerfugen'))
        &(attrs_df.Pollensammelverhalten.str.contains('polylektisch'))
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten sammeln Pollen von vielen verschiedenen Pflanzenarten und bauen ihre Nester in Mauerfugen?'
),
dict(
    conditions=(
        (attrs_df.Kuckucksbienen.str.contains('Aglaoapis'))
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten haben Bienen der Art Aglaoapis als Kuckucksbienen?'
),
dict(
    conditions=(
        (attrs_df.Familie=='Colletidae')
        &(attrs_df['Requisiten'].str.contains('Sand'))
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten der Familie Colletidae verwenden Sand als Requisiten?'
),
dict(
    conditions=(
        (attrs_df['Requisiten'].str.contains('Schnecken'))
        &(attrs_df.Kuckucksbienen.str.contains('Stelis odontopyga'))

    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten bauen ihre Nester in Schneckenhäusern und haben Bienen der Art Stelis odontopyga als Kuckucksbienen?'
),
dict(
    conditions=(

        (attrs_df['Umgangssprachliche Bezeichnungen'].str.contains('Maskenbiene'))
        &(attrs_df.Pollensammelverhalten.str.contains('breit-oligolektisch'))
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Maskenbienen zeigen ein breit-oligolektisches Pollensammelverhalten?'
),
dict(
    conditions=(
        (attrs_df['Sozialverhalten'].str.contains('solitär'))
        &(attrs_df['Nistweise'].str.contains('vorhanden'))
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten haben ein solitäres Sozialverhalten und nutzen bereits vorhandene Nistplätze?'
),
# dict(
#     conditions=(
#         #(attrs_df.Familie=='Colletidae')
#         #(attrs_df['scientific_name'].str.contains('Morawitz'))
#         #(attrs_df['Umgangssprachliche Bezeichnungen'].str.contains('Wollbiene'))
#         #(attrs_df['Kuckuckswespen'].str.contains('Sapyga'))
#         (attrs_df['Sozialverhalten'].str.contains('solitär'))
#         #& (attrs_df['Wirtsbienen'].str.contains('Andrena'))
#         &(attrs_df['Nistweise'].str.contains('vorhanden'))
#         #&(attrs_df['Baumaterial'].str.contains('Blattstücke'))
#         #(attrs_df['Requisiten'].str.contains('Schnecken'))
#         #&(attrs_df.Kuckucksbienen.str.contains('Stelis odontopyga'))
#         #&(attrs_df.Pollensammelverhalten.str.contains('breit-oligolektisch'))
#         #& (attrs_df.Baumaterial=='Blattstücke')  
#     ),
#     target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
#     question=''
# ),
]

print(len(QDQ_SAMPLES))
attrs_df[QDQ_SAMPLES[-1]['conditions']]

20


,scientific_name,Baumaterial,Familie,Kuckucksbienen,Kuckuckswespen,Nistweise,Pollensammelverhalten,Requisiten,Sozialverhalten,Umgangssprachliche Bezeichnungen,Wirtsbienen,Wissenschaftlicher Name
249,"Hylaeus duckei (Alfken, 1904)",,Colletidae,,,hypergäisch-vorhandene Hohlräume,breit-oligolektisch (Apiaceae),,solitär,Duckes Maskenbiene,,"Hylaeus duckei (Alfken, 1904)"
253,"Hylaeus hyalinatus (Smith, 1842)",,Colletidae,,,"hypergäisch-vorhandene Hohlräume, Mauerfugen, ...",polylektisch (15 Familien),,solitär,Mauer-Maskenbiene,,"Hylaeus hyalinatus (Smith, 1842)"
260,"Hylaeus paulus (Bridwell, 1919)",,Colletidae,,,hypergäisch-vorhandene Hohlräume,polylektisch (starke Präf. Rosacaceae),,solitär,Kleine Maskenbiene,,"Hylaeus paulus (Bridwell, 1919)"
262,"Hylaeus pfankuchi (Alfken, 1919)",,Colletidae,,,"hypergäisch-vorhandene Hohlräume, Gallen?",polylektisch (starke Präf. Rosacaceae),(Schilfgallen?),solitär,Ried-Maskenbiene,,"Hylaeus pfankuchi (Alfken, 1919)"
264,"Hylaeus punctatus (Brullé, 1832)",,Colletidae,,,hypergäisch-vorhandene Hohlräume,polylektisch (starke Präf. Apiaceae),,solitär,Grobpunktierte Maskenbiene,,"Hylaeus punctatus (Brullé, 1832)"
265,"Hylaeus punctulatissimus (Smith, 1842)",,Colletidae,,,hypergäisch-vorhandene Hohlräume,eng-oligolektisch (Allium),,solitär,Lauch-Maskenbiene,,"Hylaeus punctulatissimus (Smith, 1842)"
270,"Hylaeus taeniolatus (Förster, 1871)",,Colletidae,,,hypergäisch-vorhandene Hohlräume,breit-oligolektisch (Apiaceae),,solitär,Gelbhals-Maskenbiene,,"Hylaeus taeniolatus (Förster, 1871)"
271,"Hylaeus tyrolensis (Förster, 1871)",,Colletidae,,,hypergäisch-vorhandene Hohlräume,breit-oligolektisch (Apiaceae),,solitär,Tiroler Maskenbiene,,"Hylaeus tyrolensis (Förster, 1871)"
272,"Hylaeus variegatus (Fabricius, 1798)",,Colletidae,,,endogäisch-vorhandene Hohlräume,polylektisch (starke Präf. Apiaceae),,solitär,Rote Maskenbiene,,"Hylaeus variegatus (Fabricius, 1798)"


In [ ]:
#leaf trough the samples
qdq_sample=QDQ_SAMPLES.pop(0)
print(f"question: {qdq_sample['question']}")
#print(f"target_attrs: {qdq_sample['target_attrs']}")
attrs_df[qdq_sample['conditions']][qdq_sample['target_attrs']]


question: Welche Bienenarten der Megachilidae sammelen Pollen von vielen verschiedenen Pflanzenarten und bauen ihre Nester aus Blattstücken?


,scientific_name
340,"Megachile alpicola (Alfken, 1924)"
342,"Megachile apicalis (Spinola, 1808)"
343,"Megachile argentata (Fabricius, 1793)"
345,"Megachile centuncularis (Linnaeus, 1758)"
346,"Megachile circumcincta (Kirby, 1802)"
348,"Megachile genalis (Morawitz, 1880)"
349,"Megachile lagopoda (Linnaeus, 1761)"
351,"Megachile leachella (Curtis, 1828)"
352,"Megachile ligniseca (Kirby, 1802)"
353,"Megachile maritima (Kirby, 1802)"


# FFQ - Free Form Questions
- General Understanding/ General Abilities because an Answertext must be formulated

In [90]:
def compose_emb(row):
    s,p,o,so,po,spo=[row[f'{c}_emb'] for c in ['s','p','o','so','po','spo']]
    emb=po
    emb_normalized=emb/ np.linalg.norm(emb) #normalize

    compose_emb.info='po'
    return emb_normalized

passages_df=knowledge_df[knowledge_df['attr_type']=='Text'].copy()
passages_df=passages_df[['s','p','o']]
passages_df['emb']=knowledge_df.apply(compose_emb, axis=1)
passages_df.rename(
    columns={
        's':'scientific_name',
        'p':'attribute',
        'o':'passage'
        },inplace=True)
passages_df.reset_index(names='kid', inplace=True)
passages_df

,kid,scientific_name,attribute,passage,emb
0,0,"Aglaoapis tridentata (Nylander, 1848)",Aussehen,"Die Biene ist schwarz, etwa 8–10 mm lang und m...","[0.013315118, -0.0025690868, -0.002520601, -0...."
1,2,"Aglaoapis tridentata (Nylander, 1848)",Kurzbeschreibung,"Dunkle Zweizahnbiene (Aglaoapis tridentata, Sy...","[0.0050212676, -0.020270795, 0.03047962, -0.01..."
2,3,"Aglaoapis tridentata (Nylander, 1848)",Lebensraum,"Die Biene lebt auf trockenen, blütenreichen Bö...","[0.010719753, 0.0073655825, 0.023964873, -0.00..."
3,6,"Aglaoapis tridentata (Nylander, 1848)",Verbreitung,Die Biene kommt hauptsächlichen in Süd- und Mi...,"[0.018203076, -0.0093616815, 0.010161171, -0.0..."
4,11,"Amegilla quadrifasciata (de Villers, 1789)",Kurzbeschreibung,Amegilla quadrifasciata ist eine Biene aus der...,"[-0.005874074, 0.014487559, 0.025344735, -0.00..."
...,...,...,...,...,...
1039,3870,"Xylocopa iris (Christ, 1791)",Vorkommen,Die Art ist im Mittelmeerraum und östlich bis ...,"[0.018833201, 0.009829856, 0.026238477, 0.0120..."
1040,3880,"Xylocopa violacea (Linnaeus, 1758)",Kurzbeschreibung,"Die Große Blaue Holzbiene, auch Blauschwarze o...","[0.013091193, 0.0038780614, 0.022316568, 0.006..."
1041,3881,"Xylocopa violacea (Linnaeus, 1758)",Lebensweise,"Die Paarung erfolgt im Frühjahr, nachdem beide...","[-0.000986353, -0.0118575245, 0.024509192, 0.0..."
1042,3882,"Xylocopa violacea (Linnaeus, 1758)",Merkmale,Mit 20 bis 28 Millimeter Körperlänge zählen di...,"[0.013986376, 0.007970083, -0.004052453, -0.01..."


In [70]:
from sklearn.metrics.pairwise import cosine_similarity

passages_embs=np.vstack(passages_df['emb'].values)
match_matrix=cosine_similarity(passages_embs,passages_embs)
match_matrix.shape

(1044, 1044)

In [89]:
num_chunks=2

passage_row=passages_df.sample(1).iloc[0]

print(f"# {passage_row['attribute']}/ {passage_row['scientific_name']}\n")
print(f"{passage_row['passage']}")

match_index=passage_row.name
match_indices=match_matrix[match_index].argsort()[::-1][:num_chunks]
match_scores=match_matrix[match_index][match_indices]

ranking_df=passages_df.iloc[match_indices].copy()
ranking_df['score']=match_scores
ranking_df


# Merkmale/ Nomada obtusifrons (Nylander, 1848)

Die Bienen haben eine Körperlänge von fünf bis sieben Millimetern. Der Kopf und Thorax der Weibchen ist schwarz und ist rot gezeichnet. Letzterer hat manchmal auch kleine gelbliche Flecken. Die Tergite sind rot. Teilweise sind sie stark verdunkelt, teilweise haben sie weißgelbe Flecken. Das Labrum ist schwarz und hat ein kleines Zähnchen. Charakteristisch für die Art ist der abgeflachte Wulst zwischen den Fühlern, der bei ähnlichen Arten als kantiger Stirnkiel ausgebildet ist. Das Schildchen (Scutellum) ist schwarz und trägt deutliche Höcker. Die Schienen (Tibien) der Hinterbeine sind am Ende stumpf und haben vier oder fünf dicke, kurze, kleine Dornen. Die Männchen sind den Weibchen ähnlich, ihre Mandibeln und der Vorderrand der Stirnplatte sind jedoch gelb. Die Hintertibien tragen zwei bis drei kleine Dornen.


,kid,scientific_name,attribute,passage,emb,score
835,3144,"Nomada obtusifrons (Nylander, 1848)",Merkmale,Die Bienen haben eine Körperlänge von fünf bis...,"[0.015013425, -0.007370205, -0.0056248168, -0....",1.000000
853,3191,"Nomada rhenana (Morawitz, 1872)",Merkmale,Die Bienen haben eine Körperlänge von 7 bis 9 ...,"[0.019856127, -0.0008634387, -0.005509735, -0....",0.972185


In [ ]:
def get_ffq_prompt(df):

    if len(df)==1: chunk_str="chunk"
    else: chunk_str="chunks"

    prompt=""
    prompt+=f"""
You are a teacher creating evaluation questions for a student. Based on the knowledge {chunk_str} provided below, your task is to:
1. Generate a question in German that evaluates the student's understanding or ability to infer based on the knowledge.
2. Provide a "teacher's solution" in German that represents the correct answer.

Return the output in the following format as a JSON string:
{{
    "question": "Here goes the generated question in German",
    "answer": "Here goes the teacher's solution in German"
}}
""".strip()
    
    prompt+=f"\n\nKNOWLEDGE {chunk_str.upper()}:\n"
    
    for _,row in df.iterrows():
        prompt+="\n# {attribute} von {scientific_name}\n{passage}\n".format(**row)
    
    prompt+="\n\nThe output must be a valid JSON string."
    
    return prompt


prompt=get_ffq_prompt(ranking_df[:2])
pyperclip.copy(prompt)
print(prompt)

You are a teacher creating evaluation questions for a student. Based on the knowledge chunks provided below, your task is to:
1. Generate a question in German that evaluates the student's understanding or ability to infer based on the knowledge.
2. Provide a "teacher's solution" in German that represents the correct answer.

Return the output in the following format as a JSON string:
{
    "question": "Here goes the generated question in German",
    "answer": "Here goes the teacher's solution in German"
}

KNOWLEDGE CHUNKS:

# Verbreitung von Anthidium montanum (Morawitz, 1865)
Die Art ist extrem selten. Sie kommt auf den Gebirgen Europas (auf den Pyrenäen, den Alpen, dem Schwarzwald, dem Erzgebirge, den Karpaten, dem Krimgebirge) und Zentralasiens vor. Im deutschsprachigen Raum fast ausschließlich in den Alpen. Es gibt einen aktuellen Nachweis aus der Schwäbischen Alb, sonst überwiegend historische Funde.

# Vorkommen von Xylocopa iris (Christ, 1791)
Die Art ist im Mittelmeerraum und

In [86]:
output={
    "question": "Welche Gemeinsamkeiten und Unterschiede gibt es in der Verbreitung von Anthidium montanum und Xylocopa iris? Begründen Sie Ihre Antwort anhand der gegebenen Informationen.",
    "answer": "Gemeinsamkeiten: Beide Arten sind selten und haben ein Verbreitungsgebiet, das teilweise in Europa und Zentralasien liegt. Unterschiede: Anthidium montanum kommt vor allem in Gebirgsregionen vor, während Xylocopa iris vor allem im Mittelmeerraum verbreitet ist. In Deutschland findet man Anthidium montanum fast nur in den Alpen und einen Nachweis in der Schwäbischen Alb, während Xylocopa iris nur in Baden-Württemberg vorkommt. Zusätzlich ist Xylocopa iris in der Schweiz stark gefährdet, während für Anthidium montanum keine solche Gefährdung erwähnt wird."
}


In [ ]:
ffq_sample=dict(
    df=ranking_df,
    compose_emb_info=compose_emb.info,
    prompt=prompt
    output=
)

ffq_sample

{'id': 2923,
 'scientific_name': 'Nomada errans (Lepeletier, 1841)',
 'header': 'Vorkommen und Lebensweise',
 'passage': 'Die Art ist in Süd- und mancherorts auch in Mitteleuropa und in England verbreitet. Die Tiere fliegen von Mitte Juni bis Mitte August. Die Art parasitiert Andrena nitidiuscula und vielleicht auch Andrena pallitarsis.'}

In [21]:
ffqs_df=pd.DataFrame()
ffqs_df

""


# Experiment

In [77]:
trainwrap=LoraTrainerWrap()
trainwrap.get_config()
trainwrap.spawn()

2025-02-03 12:44:42,042 - __init__.py - beesup_llm - __init__ - INFO - 0001_LORA_TRAINER initialised
2025-02-03 12:44:42,053 - __init__.py - beesup_llm - spawn_config - INFO - 0001_LORA_TRAINER config spawned at injection_lab/lora_trainers/0001_lora_trainer


In [ ]:
experiment=InjectionExperiment()